This notebook pulls 2010 census data and attempts to generate a large set of points that approximates a smooth surface

In [23]:
# Declare static variables

n=50 # The number of points to assign to each census block

In [24]:
# Import libraries

import pandas as pd
import geopandas
import numpy as np
import requests
from io import BytesIO
import folium
from IPython.display import clear_output

In [59]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_10_140_00_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract_data = geopandas.read_file(BytesIO(data.content))

In [60]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_10_tabblock10.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
block_data = geopandas.read_file(BytesIO(data.content))

In [27]:
# For each census block, create a bounding box
block_bounds = block_data["geometry"].bounds

# Attch GEOID to boundaries
block_bounds = block_data[["GEOID10","geometry"]].merge(block_bounds, left_index=True, right_index=True)

In [28]:
# Fit a 2D Gaussian distribution over the bounding boxes

# Takes in a row of 'block_bounds' and outputs a 2D Gaussian distribution of 'n' points over the bounding box, as well as the GEOID
def get_points(row,n):
    print(f"Processing Block {row['GEOID10']}...")
    # 'i' is the total number of points left to assign
    i=n
    # 'points_return' is the list of all points for the block
    # TODO: CRS is hardcoded
    points_return = geopandas.GeoSeries(crs="EPSG:4269")
    # Allocate points until n have been assigned
    while i > 0:
        # Generates a Gaussian distribution for the y-axis, located at the center of the box with standard deviation such that 95% of the distribution lies in the bounding box
        pointsy = np.random.normal(loc=(row["maxy"] + row["miny"])/2, scale=np.abs(row["maxy"] - row["miny"])/1.96, size=i)
        # Generates a Gaussian distribution for the x-axis, located at the center of the box with standard deviation such that 95% of the distribution lies in the bounding box
        pointsx = np.random.normal(loc=(row["maxx"] + row["minx"])/2, scale=np.abs(row["maxx"] - row["minx"])/1.96, size=i)
        # Convert the points to Shapely points
        points = geopandas.GeoSeries(geopandas.points_from_xy(pointsx, pointsy, crs="EPSG:4269"))
        # Check if the points are inside the block
        point_checks = points.within(row["geometry"])
        # Add found points to our list
        points_return = geopandas.GeoSeries(pd.concat([points_return, points[point_checks]], ignore_index=True), crs=points_return.crs)
        # Set 'i' equal to the number of missed points
        i = n - points_return.size
    
    # Clear warnings from notebook output to prevent crash
    clear_output()
    # Return an array with every point in the cloud and the GEOID
    return np.append(points_return.values, row["GEOID10"])
    

In [29]:
# Fit a Gaussian distribution to each block
point_cloud = block_bounds.apply(get_points, axis=1, args=(n,), result_type='expand')

# Rename columns of the pointcloud
point_cloud.columns = ['GEOID' if str(x)==str(n) else 'point_' + str(x) for x in point_cloud.columns]

In [61]:
# Pull population data for 2010 Census blocks
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'block:*' # for predicate
IN = 'state:10&in=county:*&in=tract:*'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATEFP10", "county":"COUNTYFP10", "tract":"TRACTCE10", "block":"BLOCKCE10"}, inplace=True)

# Attach to block shapes
block_data = block_data.merge(data, on=["STATEFP10","COUNTYFP10","TRACTCE10","BLOCKCE10"])

In [62]:
# Pull population data for 2010 Census tracts
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATE", "county":"COUNTY", "tract":"TRACT"}, inplace=True)

# Attach to tract shapes
tract_data = tract_data.merge(data, on=["STATE","COUNTY","TRACT"])

In [64]:
# Assign a fraction of the population of each block as a value to each point
point_cloud["population_per_point"] = block_data["P001001"].astype(int) / n

In [65]:
# Flatten to a GeoSeries where each row is a point and its weight
weights = np.array([[row["population_per_point"] for _ in range(n)] for i, row in point_cloud.iterrows()]).flatten()
points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in point_cloud.iterrows()]).flatten()
points_list = geopandas.GeoDataFrame({"population_per_point":weights,"geometry":points}, crs="EPSG:4269")

# Determine the number of points in the point cloud. This should be n * the number of census blocks
print(points_list.shape[0] / n == block_data.shape[0])

/Users/emiller/opt/anaconda3/envs/census-tract-redistricting/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1308/4162940238.py:3: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in point_cloud.iterrows()]).flatten()
/var/folders/9r/738fljg52

True


In [90]:
# TODO: Replace with spatial join

# For each point, multiplies variables of interest in the containing census tract by population weight and returns results
# TODO: Approximately 1% of points are being missed
def assign_to_points(row, tracts):
    checks = tracts["geometry"].contains(row["geometry"])
    if checks.any():
        containing_tract = tracts.loc[checks]
        output = containing_tract[["P001001","H001001","P001003"]].astype(int) / containing_tract["P001001"].astype(int).values[0] * row["population_per_point"]
        output["geometry"] = row["geometry"]
        return output.values[0]
    else:
        print("Missed a point!")
        return [0,0,0,row["geometry"]]

In [91]:
# Transfer data from 2010 census tracts to smooth surface
variables_per_point = points_list.apply(assign_to_points, axis=1, args=(tract_data,), result_type='expand')
#clear_output()
print(variables_per_point)
# Add descriptive column names
variables_per_point.columns = ["P001001","H001001","P001003","geometry"]
# Cast to a GeoDataFrame
variables_per_point = geopandas.GeoDataFrame(variables_per_point)

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

...

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a point!
Missed a

In [92]:
# Request shapefile data for 2020 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_10_tract_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract2020 = geopandas.read_file(BytesIO(data.content))

In [93]:
# TODO: Replace with spatial join

# For each 2020 census tract, find all points within the tract and sum their values. Divide each variable of interest by the summed population to get estimates
def sum_to_tract(row, points):
    # Test each point for containment in the tract
    checks = points["geometry"].within(row["geometry"])
    # If there are any points in the tract, sum the points
    if checks.any():
        # Locate all points in the tract
        points_in_tract = points.loc[checks]
        # Sum all points in the tract
        summed_var = points_in_tract[["P001001","H001001","P001003"]].sum()
        # Add the GEOID of the tract as a column
        summed_var["GEOID"] = row["GEOID"]
        return summed_var.values
    else:
        print("Tract contains no points!")
        return [0,0,0, row["GEOID"]]
    
    

In [94]:
# Sum data from points to 2020 census tracts
interpolated_values = tract2020.apply(sum_to_tract, axis=1, args=(variables_per_point,), result_type='expand')

In [95]:
# Pull population data for 2020 Census tracts
# Define request parameters

year = '2020' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P1_001N,H1_001N,P1_003N' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
tract2020_data = pd.DataFrame(data[1:], columns = data[0])

# Add a GEOID column to the data
tract2020_data["GEOID"] = tract2020_data["state"].astype(str) + tract2020_data["county"].astype(str) +tract2020_data["tract"].astype(str)

In [96]:
# Write combined dataframe of 2020 ground truth and estimated values to a csv
interpolated_values.merge(tract2020_data, left_on=3, right_on="GEOID").to_csv("estimates.csv", index=False)

In [97]:
interpolated_values.merge(tract2020_data, left_on=3, right_on="GEOID")

,0,1,2,3,P1_001N,H1_001N,P1_003N,state,county,tract,GEOID
0,2880.84,1136.829207,1915.189682,10003010300,4106,1782,1968,10,003,010300,10003010300
1,2372.28,1024.110842,2099.649667,10003011300,2244,960,1789,10,003,011300,10003011300
2,4988.26,1673.255348,3121.823689,10003013903,4820,1670,2443,10,003,013903,10003013903
3,3189.70,1118.911101,221.177111,10003015400,2932,1142,237,10,003,015400,10003015400
4,1874.28,791.872145,1621.565958,10003013608,1959,802,1453,10,003,013608,10003013608
...,...,...,...,...,...,...,...,...,...,...,...
254,4645.84,1729.519840,3154.253920,10001040204,6451,2269,3613,10,001,040204,10001040204
255,2361.92,1548.579257,2076.439900,10005050808,4150,2478,3458,10,005,050808,10005050808
256,6545.50,2470.721291,5706.091023,10001040100,7315,2740,5980,10,001,040100,10001040100
257,1688.16,1277.978132,1636.494114,10005051307,2975,2649,2767,10,005,051307,10005051307


In [69]:
tract_data

,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,geometry,P001001,H001001,P001003
0,1400000US10001042800,10,001,042800,428,Tract,57.104,"POLYGON ((-75.73286 38.95737, -75.73845 39.027...",7085,2972,5743
1,1400000US10001042900,10,001,042900,429,Tract,26.924,"POLYGON ((-75.48935 38.90583, -75.48939 38.905...",4794,1924,3902
2,1400000US10001043100,10,001,043100,431,Tract,59.523,"POLYGON ((-75.72400 38.84672, -75.72406 38.847...",2630,1047,2441
3,1400000US10001043300,10,001,043300,433,Tract,2.967,"POLYGON ((-75.52838 39.16143, -75.52827 39.161...",6131,1743,2225
4,1400000US10001043400,10,001,043400,434,Tract,25.532,"POLYGON ((-75.48949 38.90550, -75.48939 38.905...",4146,1752,3207
...,...,...,...,...,...,...,...,...,...,...,...
213,1400000US10005051202,10,005,051202,512.02,Tract,1.487,"POLYGON ((-75.05625 38.56308, -75.05486 38.546...",926,1845,912
214,1400000US10005051204,10,005,051204,512.04,Tract,1.002,"POLYGON ((-75.05363 38.53216, -75.05359 38.531...",710,3050,698
215,1400000US10005051205,10,005,051205,512.05,Tract,1.412,"POLYGON ((-75.04931 38.46747, -75.04906 38.456...",699,1841,679
216,1400000US10005051301,10,005,051301,513.01,Tract,13.239,"POLYGON ((-75.15542 38.59404, -75.14132 38.597...",4936,3736,4785
